## Task 1 - SQL

### Build SparkSession:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip3 install pyspark==3.0.2
!pip3 install findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop2.7"

     |████████████████████████████████| 204.8 MB 47 kB/s 
     |████████████████████████████████| 198 kB 66.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.2-py2.py3-none-any.whl size=205186690 sha256=5745defe5c50451334fe031ba97af99cddf162fe8ee7ffd4c73d0e540ba89861
  Stored in directory: /root/.cache/pip/wheels/9a/39/f6/970565f38054a830e9a8593f388b36e14d75dba6c6fdafc1ec
Successfully built pyspark


In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = (SparkSession.builder.appName('SparkSQL')
         .enableHiveSupport()
         .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.0.1") 
         .getOrCreate())

spark = SparkSession.builder.appName('SparkSQL').enableHiveSupport().getOrCreate()

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving DataFrames_sample.json to DataFrames_sample.json
Saving ITI_data - ITI_data.csv to ITI_data - ITI_data.csv
Saving test3 - test3.csv to test3 - test3.csv
Saving test1 - test1.csv to test1 - test1.csv


### Read the json file:

In [ ]:
jdf = spark.read.json("DataFrames_sample.json")
jdf.show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
| 8.0|20.3|  1TB SSD|  4|       iMac|64GB|       27"| 25.6|  20.8|2017|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Display the schema:


In [ ]:
jdf.printSchema()

root
 |-- D: double (nullable = true)
 |-- H: double (nullable = true)
 |-- HDD: string (nullable = true)
 |-- Id: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- RAM: string (nullable = true)
 |-- ScreenSize: string (nullable = true)
 |-- W: double (nullable = true)
 |-- Weight: double (nullable = true)
 |-- Year: long (nullable = true)



### Get all the data when "Model" equal "MacBook Pro":




In [ ]:
jdf.select('*').where("Model == 'MacBook Pro'").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Create TempView:

In [ ]:
jdf.createOrReplaceTempView("models")

### Display "RAM"column and count "RAM" column:

In [ ]:
spark.sql("""SELECT RAM FROM models """).show()

+----+
| RAM|
+----+
|16GB|
| 8GB|
| 8GB|
|64GB|
+----+



In [ ]:
spark.sql("""SELECT COUNT(RAM) FROM models """).show()

+----------+
|count(RAM)|
+----------+
|         4|
+----------+



### Get all columns when "Year" column equal "2015"  

In [ ]:
spark.sql("""SELECT * FROM models WHERE year = 2015""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all when "Model" start with "M":

In [ ]:
spark.sql("""SELECT * FROM models WHERE Model LIKE 'M%'""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
|7.74|0.52|256GB SSD|  2|    MacBook| 8GB|       12"|11.04|  2.03|2016|
|8.94|0.68|128GB SSD|  3|MacBook Air| 8GB|     13.3"| 12.8|  2.96|2016|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data when "Model" column equal "MacBook Pro"

In [ ]:
spark.sql("""SELECT * FROM models WHERE Model == 'MacBook Pro'""").show()

+----+----+---------+---+-----------+----+----------+-----+------+----+
|   D|   H|      HDD| Id|      Model| RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-----------+----+----------+-----+------+----+
|9.48|0.61|512GB SSD|  1|MacBook Pro|16GB|       15"|13.75|  4.02|2015|
+----+----+---------+---+-----------+----+----------+-----+------+----+



### Get all data with Multiple Conditions when "RAM" column equal "8GB" and "Model" column is "Macbook".

In [ ]:
spark.sql("""SELECT * FROM models WHERE RAM == '8GB' AND Model == 'MacBook'""").show()

+----+----+---------+---+-------+---+----------+-----+------+----+
|   D|   H|      HDD| Id|  Model|RAM|ScreenSize|    W|Weight|Year|
+----+----+---------+---+-------+---+----------+-----+------+----+
|7.74|0.52|256GB SSD|  2|MacBook|8GB|       12"|11.04|  2.03|2016|
+----+----+---------+---+-------+---+----------+-----+------+----+



### Get all data with Multiple Conditions when "D" greater than or equal "8" and "Model" column is "iMac".

In [ ]:
spark.sql("""SELECT * FROM models WHERE D >= 8 AND Model == 'iMac'""").show()

+---+----+-------+---+-----+----+----------+----+------+----+
|  D|   H|    HDD| Id|Model| RAM|ScreenSize|   W|Weight|Year|
+---+----+-------+---+-----+----+----------+----+------+----+
|8.0|20.3|1TB SSD|  4| iMac|64GB|       27"|25.6|  20.8|2017|
+---+----+-------+---+-----+----+----------+----+------+----+



## Task 2


### Read "test1" dataset:

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, FloatType

schema = StructType([ \
    StructField("Name",StringType(),True), \
    StructField("age",FloatType(),True), \
    StructField("Experience", FloatType(), True), \
    StructField("Salary", FloatType(), True) \
  ])

tdf = spark.read.csv("test1 - test1.csv", schema=schema, header=True)
tdf.show()

+---------+----+----------+-------+
|     Name| age|Experience| Salary|
+---------+----+----------+-------+
|    Krish|31.0|      10.0|30000.0|
|Sudhanshu|30.0|       8.0|25000.0|
|    Sunny|29.0|       4.0|20000.0|
|     Paul|24.0|       3.0|20000.0|
|   Harsha|21.0|       1.0|15000.0|
|  Shubham|23.0|       2.0|18000.0|
+---------+----+----------+-------+



In [ ]:
tdf.createOrReplaceTempView("test1")

### Display Salary of the people less than or equal to 20000

In [ ]:
tdf.select('Name', 'Salary').where(tdf.Salary <= 20000).show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
| Harsha|15000.0|
|Shubham|18000.0|
+-------+-------+



In [ ]:
spark.sql("""SELECT Name, Salary FROM test1 WHERE Salary <= 20000""").show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
| Harsha|15000.0|
|Shubham|18000.0|
+-------+-------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [ ]:
tdf.select('Name','Salary').where((tdf.Salary <= 20000) & (tdf.Salary >= 15000)).show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
| Harsha|15000.0|
|Shubham|18000.0|
+-------+-------+



In [ ]:
spark.sql("""SELECT Name, Salary FROM test1 WHERE Salary <= 20000 AND Salary >= 15000""").show()

+-------+-------+
|   Name| Salary|
+-------+-------+
|  Sunny|20000.0|
|   Paul|20000.0|
| Harsha|15000.0|
|Shubham|18000.0|
+-------+-------+



## Task 3 

### Read "test3" dataset:

In [ ]:
schema = StructType([ \
    StructField("Name",StringType(),True), \
    StructField("Departments",StringType(),True), \
    StructField("Salary", FloatType(), True) \
  ])

ddf = spark.read.csv("test3 - test3.csv", header=True, schema=schema)

In [ ]:
ddf.createOrReplaceTempView("test3")

### Display dataset

In [ ]:
ddf.show()

+---------+------------+-------+
|     Name| Departments| Salary|
+---------+------------+-------+
|    Krish|Data Science|10000.0|
|    Krish|         IOT| 5000.0|
|   Mahesh|    Big Data| 4000.0|
|    Krish|    Big Data| 4000.0|
|   Mahesh|Data Science| 3000.0|
|Sudhanshu|Data Science|20000.0|
|Sudhanshu|         IOT|10000.0|
|Sudhanshu|    Big Data| 5000.0|
|    Sunny|Data Science|10000.0|
|    Sunny|    Big Data| 2000.0|
+---------+------------+-------+



### Display schema

In [ ]:
ddf.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- Salary: float (nullable = true)



### Group by "Name" column and using sum function on "Name" column

#### Can't use sum on name column so I added two possible solutions.

In [ ]:
ddf.groupBy('Name').count().show()

+---------+-----+
|     Name|count|
+---------+-----+
|Sudhanshu|    3|
|    Sunny|    2|
|    Krish|    3|
|   Mahesh|    2|
+---------+-----+



In [ ]:
spark.sql("""SELECT Name, COUNT(Name) FROM test3 GROUP BY Name""").show()

+---------+-----------+
|     Name|count(Name)|
+---------+-----------+
|Sudhanshu|          3|
|    Sunny|          2|
|    Krish|          3|
|   Mahesh|          2|
+---------+-----------+



In [ ]:
ddf.groupBy('Name').sum().show()

+---------+-----------+
|     Name|sum(Salary)|
+---------+-----------+
|Sudhanshu|    35000.0|
|    Sunny|    12000.0|
|    Krish|    19000.0|
|   Mahesh|     7000.0|
+---------+-----------+



In [ ]:
spark.sql("""SELECT Name, SUM(Salary) FROM test3 GROUP BY Name""").show()

+---------+-----------+
|     Name|sum(Salary)|
+---------+-----------+
|Sudhanshu|    35000.0|
|    Sunny|    12000.0|
|    Krish|    19000.0|
|   Mahesh|     7000.0|
+---------+-----------+



### Group by "Name" column and using avg function on "Name" column

In [ ]:
ddf.groupBy('Name').avg('Salary').show()

+---------+------------------+
|     Name|       avg(Salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



In [ ]:
spark.sql("""SELECT Name, AVG(Salary) FROM test3 GROUP BY Name""").show()

+---------+------------------+
|     Name|       avg(Salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [ ]:
ddf.groupBy('Departments').sum('Salary').show()

+------------+-----------+
| Departments|sum(Salary)|
+------------+-----------+
|         IOT|    15000.0|
|    Big Data|    15000.0|
|Data Science|    43000.0|
+------------+-----------+



In [ ]:
spark.sql("""SELECT Departments, SUM(Salary) FROM test3 GROUP BY Departments""").show()

+------------+-----------+
| Departments|sum(Salary)|
+------------+-----------+
|         IOT|    15000.0|
|    Big Data|    15000.0|
|Data Science|    43000.0|
+------------+-----------+



### Group by "Departments" column and using mean function on "Departments" column:

In [ ]:
ddf.groupBy('Departments').mean('Salary').show()

+------------+-----------+
| Departments|avg(Salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



In [ ]:
spark.sql("""SELECT Departments, MEAN(Salary) FROM test3 GROUP BY Departments""").show()

+------------+------------+
| Departments|mean(Salary)|
+------------+------------+
|         IOT|      7500.0|
|    Big Data|      3750.0|
|Data Science|     10750.0|
+------------+------------+



Group by "Departments" column and using count function on "Departments" column:

In [ ]:
ddf.groupBy('Departments').count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



In [ ]:
spark.sql("""SELECT Departments, COUNT(Departments) FROM test3 GROUP BY Departments""").show()

+------------+------------------+
| Departments|count(Departments)|
+------------+------------------+
|         IOT|                 2|
|    Big Data|                 4|
|Data Science|                 4|
+------------+------------------+



### Apply agg to using sum function get the total of salaries

In [ ]:
ddf.agg({'Salary':'sum'}).show()

+-----------+
|sum(Salary)|
+-----------+
|    73000.0|
+-----------+



In [ ]:
spark.sql("""SELECT SUM(Salary) FROM test3""").show()

+-----------+
|sum(Salary)|
+-----------+
|    73000.0|
+-----------+



## Task 4

You've been flown to their headquarters in Ulsan, South Korea, to assist them in accurately estimating the number of crew members a ship will need.


They're currently building new ships for certain customers, and they'd like you to create a model and utilize it to estimate how many crew members the ships will require.


Metadata:
1. Measurements of ship size 
2. capacity 
3. crew 
4. age for 158 cruise ships.

It is saved in a csv file for you called "ITI_data.csv". our task is to develop a regression model that will assist in predicting the number of crew members required for future ships. The client also indicated that they have found that particular cruise lines will differ in acceptable crew counts, thus this is most likely an important factor to consider when conducting your investigation.

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql.types import StructType,StructField, StringType, FloatType
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression

In [ ]:
schema = StructType([ \
    StructField("Ship_name",StringType(),True), \
    StructField("Cruise_line",StringType(),True), \
    StructField("Age",FloatType(),True), \
    StructField("Tonnage", FloatType(), True), \
    StructField("passengers", FloatType(), True), \
    StructField("length", FloatType(), True), \
    StructField("cabins", FloatType(), True), \
    StructField("passenger_density", FloatType(), True), \
    StructField("crew", FloatType(), True)
  ])

mdf = spark.read.csv("ITI_data - ITI_data.csv", header=True, schema=schema)

In [ ]:
mdf.show()

+-----------+-----------+----+-------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line| Age|Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+----+-------+----------+------+------+-----------------+----+
|    Journey|    Azamara| 6.0| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara| 6.0| 30.277|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival|26.0| 47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival|11.0|  110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival|17.0|101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival|22.0| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival|15.0| 70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival|23.0| 70.367|     20.56|  8.55| 10.22|            34.23| 9.2|
|Fascination|   Carnival|19.0| 70.367|     

In [ ]:
mdf.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- Tonnage: float (nullable = true)
 |-- passengers: float (nullable = true)
 |-- length: float (nullable = true)
 |-- cabins: float (nullable = true)
 |-- passenger_density: float (nullable = true)
 |-- crew: float (nullable = true)



In [ ]:
trainDF, testDF = mdf.randomSplit([.8,.2],seed=42)
print(f"There are {trainDF.count()} rows in the training set, and {testDF.count()} in the test set")

There are 133 rows in the training set, and 25 in the test set


### OneHotEncoder 


In [ ]:
categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]
categoricalCols

['Ship_name', 'Cruise_line']

In [ ]:
indexOutputCols = [x + "_Index" for x in categoricalCols]
indexOutputCols

['Ship_name_Index', 'Cruise_line_Index']

In [ ]:
oheOutputCols = [x + "_OHE" for x in categoricalCols]
oheOutputCols

['Ship_name_OHE', 'Cruise_line_OHE']

In [ ]:
stringIndexer = StringIndexer(inputCols=categoricalCols,
                             outputCols=indexOutputCols,
                             handleInvalid='skip')
oheEncoder = OneHotEncoder(inputCols=indexOutputCols,
                          outputCols=oheOutputCols)

In [ ]:
numericCols = [field for (field,dataType) in trainDF.dtypes
              if ((dataType=='float')& (field!='crew'))]
numericCols

['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density']

In [ ]:
assemblerInputs = oheOutputCols + numericCols
assemblerInputs

['Ship_name_OHE',
 'Cruise_line_OHE',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density']

###Use VectorAssembler to merge all columns into one column:

In [ ]:
vecAssembler = VectorAssembler(inputCols=assemblerInputs,outputCol='features')

### Create a Linear Regression Model 

In [ ]:
lr = LinearRegression(featuresCol='features',labelCol='crew')

### Creating a Pipeline

In [ ]:
pipeline =Pipeline(stages = [stringIndexer,oheEncoder,vecAssembler,lr])

In [ ]:
pipelineModel = pipeline.fit(trainDF)

In [ ]:
predDF = pipelineModel.transform(testDF)

In [ ]:
predDF.select('features','crew','prediction').show(5)

+--------------------+----+-------------------+
|            features|crew|         prediction|
+--------------------+----+-------------------+
|(141,[26,119,135,...|12.0| 14.847485844337372|
|(141,[49,118,135,...|8.69|  7.525302597076718|
|(141,[89,121,135,...| 6.3|  6.256179021691103|
|(141,[9,134,135,1...|0.88|-2.8972324122440165|
|(141,[112,133,135...|1.97|-2.3473271524128823|
+--------------------+----+-------------------+



### Model Evaluation

In [ ]:
# Using RMSE
from pyspark.ml.evaluation import RegressionEvaluator

regressionEvaluator = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='rmse')
rmse = regressionEvaluator.evaluate(predDF)
print(f"RMSE is {rmse:.1f}")

RMSE is 2.9


In [ ]:
# Using R^2
r2 = RegressionEvaluator(predictionCol='prediction',
                                         labelCol='crew',
                                         metricName='r2').evaluate(predDF)
print(f"R2 is {r2}")

R2 is 0.5060007872277537


By Eng. Mostafa Nabieh 
If you have questions, please feel free to ask.

My Email : nabieh.mostafa@yahoo.com

My Whatsapp : +201015197566